#Ingestion del archivo country.json


### Paso 1 - Leer el archivo JSON usando "DataframeReader"

In [0]:
dbutils.widgets.text("p_environment", "")
v_enviroment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run 
"../includes/common_functions"

In [0]:
%run
"../includes/configuration"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType

In [0]:
name_schema = StructType(fields=[
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
person_schema = StructType(fields=[
    StructField("personId", IntegerType(), True),
    StructField("personName", name_schema, True)
])

In [0]:
person_df = spark.read \
    .schema(person_schema)\
    .json(f"{bronze_folder_path}/{v_file_date}/person.json")



## Parte 2 : Renombrar las columnas y añadir nuevas columnas
##### 1. "personId" renombrar a "person_id"
##### 2. Agregar las columnas "Ingestion_date" y "environment"
##### 3. Agregar la columna "name" a partir de la concatenación de "forename" y "surname"

In [0]:
from pyspark.sql.functions import col,concat,current_timestamp,lit



In [0]:
person_with_columns_df = person_df \
    .withColumnRenamed("personId", "person_id") \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("name", concat(col("personName.forename"), lit(" "), col("personName.surname"))) \
    .withColumn("enviroment", lit("Production")) \
    .withColumn("file_date", lit(v_file_date))





In [0]:
person_final_df = person_with_columns_df \
    .drop("personName")


### Paso 3. Salida en un formato parquet

In [0]:
#overwrite_partition (person_final_df, "movie_silver", "person", "file_date")

In [0]:
person_final_df.write.mode("overwrite").partitionBy("file_date").format("delta").saveAsTable("movie_silver.person")

In [0]:
%sql

select file_date,count(1)
from movie_silver.person
group by file_date


In [0]:
dbutils.notebook.exit("Success No 5")